In [1]:
import numpy as np
import sputter as sp
import matplotlib.pyplot as pl
import everest as ev
import csv
from numpy import genfromtxt
from everest import detrender
from everest.math import SavGol, Scatter, Downbin
from astropy.stats import median_absolute_deviation as mad
from numpy.polynomial import polynomial as P
from tqdm import tqdm
from scipy.optimize import curve_fit
% matplotlib qt
# % matplotlib inline

In [9]:
datapath = '/Users/nks1994/Documents/Research/everest/docs/c0'
dataloc = '.csv'

# CAMPAIGN 1

tags = []
mags = []
with open((datapath+str(1)+dataloc),'r') as f:
    data = csv.reader(f)
    for i,row in enumerate(data):
        if i == 0:
            continue
        else:
            tags.append(row[0])
            mags.append(row[1])

In [ ]:
cdpps = []
for t in tags:
    star = ev.Everest(int(t))
    cdpps.append(star.cdpp)

In [ ]:
cdpps = np.load('k21cdpps.npy')
length = len(cdpps)

In [ ]:
fig = pl.figure(figsize=(15,5))
pl.plot(mags[:length],cdpps,'b.',alpha = 0.25)
pl.ylim([0,700])
pl.xlim([5,20])

In [ ]:
#def rawCDPP(mag,motion):
mag = 10
motion = 0
star = np.load('stars/larger_aperture/mag'+str(mag)+'motion'+str(motion)+'.npz')
fpix = star['fpix']


In [ ]:
def simCDPP(mag,motion):

    MN = sp.MotionNoise()
    flux,rawflux = MN.DetrendFpix(mag,motion)

    CDPP = MN.CDPP(rawflux)

    return CDPP

In [ ]:
rawcdpps = []
for i in tqdm(range(5)):
    n = i+10
    rawcdpps.append(simCDPP(n,1))

In [ ]:
simcdpps = []
for i in tqdm(range(5)):
    n = i + 10
    simcdpps.append(simCDPP(n,1))

In [ ]:
fig = pl.figure(figsize=(15,5))
pl.plot(mags[:length],cdpps,'b.',alpha = 0.25)
pl.ylim([0,6000])
pl.xlim([5,20])
xs = [i+10 for i in range(5)]
pl.plot(xs,rawcdpps,'g.',markersize=12)
pl.plot(xs,simcdpps,'r.',markersize=12)
pl.xlabel(r'$K_p\ Mag$')
pl.ylabel(r'$CDPP$')

In [10]:
t = np.linspace(0,20,1399)

from scipy.optimize import curve_fit

mags = [float(i) for i in mags] 

def fitFunc(x, a, b, c):
    return a * np.exp(b*x) + c

In [ ]:
fitParams, fitCovariance = curve_fit(fitFunc, t, cdpps, p0=(1,1e-5,1))

In [ ]:
fig = pl.figure(figsize=(15,5))
sigma = [fitCovariance[0,0], \
         fitCovariance[1,1], \
         ]
pl.plot(t, fitFunc(t, fitParams[0], fitParams[1], fitParams[2]))
#        t, fitFunc(t, fitParams[0] + sigma[0], fitParams[1] - sigma[1], fitParams[2] + sigma[2]),\
#        t, fitFunc(t, fitParams[0] - sigma[0], fitParams[1] + sigma[1], fitParams[2] - sigma[2]))

In [ ]:
pl.plot(mags[:length],cdpps,'k.')

In [ ]:
msort = sorted(mags,key=int)
d = {k:m for k,m in enumerate(msort[:length])}
csort = cdpps.sorted(key=d.get)

In [ ]:
csort = cdpps.reshape(mags[:length])

In [ ]:
motion = 0
mag = 14
path = 'stars/larger_aperture/mag' + str(mag) + 'motion' + str(motion) + '.npz'
fpix = np.load(path)['fpix']

fpix_rs = fpix.reshape(len(fpix),-1)
tempflux = np.sum(fpix_rs,axis=1)

In [ ]:
pl.plot(tempflux,'k.')

In [ ]:
import everest

In [ ]:
everest.missions.k2.CDPP(tempflux[tempflux>1000000])

In [ ]:
MN = sp.MotionNoise()
MN.CDPP(tempflux)

In [ ]:
star = everest.Everest(205071984)

In [ ]:
star.cdpp

In [ ]:
star.mag

In [ ]:
star.cdppr

In [11]:
counts = []
N = []
for t in tags[:10]:
    t = int(t)
    star = everest.Everest(t)
    counts.append(np.mean(star.apply_mask(star.flux)))
    N.append((np.std(star.apply_mask(star.flux))/np.sqrt(np.mean(star.apply_mask(star.flux)))))

NameError: name 'everest' is not defined

In [21]:
Afile = np.load('A.npz')
A = Afile['A']

Aguess = [1e7,1,-8]

In [22]:

def func(x, a, b, x0):
    return a * np.exp(-b * (x+x0))

# Simulate data
counts = np.random.random(1000) * 200000
N = func(counts, 0.1, 1. / 2500, 0) + 0.0025 * np.random.randn(1000)

# Plot data
pl.plot(counts, N, 'r.', alpha = 0.3, ms = 2)

# Fit the exponential
guess = [0.1, 1. / 2500, 0]
coeffs, _ = curve_fit(func, counts, N, p0 = guess)

# Plot the fit
xfit = np.linspace(0, 200000, 1000)
pl.plot(xfit, func(xfit, *coeffs), 'k-')

# Show
pl.show() 

In [23]:
Acoeffs, _ = curve_fit(func, mags[:len(A)], A, p0 = Aguess)
Acoeffs

array([  1.98199852e+07,   9.13498171e-01,  -7.14784609e+00])

In [41]:
fig = pl.figure(figsize=(15,5))
x_ = np.linspace(0, 20, 1000)
pl.plot(mags[:len(A)],A,'r.',alpha=0.5)
pl.plot(x_,func(x_, 1.65e+07, 0.93, -7.35),'b')
#pl.xlim([8,20])
pl.ylim([0,1e7])
pl.show()

In [36]:
def amp(mag):
    x=mag
    a,b,c = 1.65e+07, 0.93, -7.35
    return a * np.exp(-b * (x+c)) 

In [37]:
amp(12)

218462.78067996842

In [38]:
def Amplitude(self, mag):
    '''
    Returns the amplitude of the PSF for a star of a given magnitude.
    '''

    x = mag
    a,b,c = 1.65e+07, 0.93, -7.35
    return a * np.exp(-b * (x+c)) 

In [40]:
len(tags)

18942

In [54]:
xpl = np.linspace(0,1,100)
xpix = np.linspace(-.5,.5,100)

In [43]:
def PSF(params):
    '''
    Create PSF model from params
    '''
    amp1,amp2,x01,x02,y01,y02,background = params

    cx_1 = [1.,0.,-0.3]
    cx_2 = [1.,0.,-0.3]
    cy_1 = [1.,0.,-0.3]
    cy_2 = [1.,0.,-0.3]

    sx = 0.5
    sy = 0.5
    rho = 0.0


    model = np.zeros((9,9))
    for i in range(9):
        for j in range(9):
            model[i][j] = PixelFlux(cx_1, cy_1, [amp1], [x01-i], [y01-j], [sx], [sy], [rho]) + \
                          PixelFlux(cx_2, cy_2, [amp2], [x02-i], [y02-j], [sx], [sy], [rho]) + \
                          background
    return model

In [55]:
PS = lambda x: 1. -0.3 * x**2

In [62]:
pl.plot(xpl,PS(xpix),'k')
pl.ylim([.8,1.])
pl.xlabel('x (pixels)')
pl.ylabel('Sensitivity')
pl.show()